In [1]:
import numpy as np 
import pandas as pd
from pathlib import Path
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [2]:
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [3]:
if iskaggle:
    path = Path('../input/nlp-getting-started')
    ! pip install -q datasets


In [4]:
df = pd.read_csv(path/'train.csv')


In [5]:
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
df.describe(include=[object])

,keyword,location,text
count,7552,5080,7613
unique,221,3341,7503
top,fatalities,USA,11-Year-Old Boy Charged With Manslaughter of T...
freq,45,104,10


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [8]:
mode = df.mode().iloc[0]

In [9]:
df.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [10]:
df.fillna(mode,inplace=True)

In [11]:
df.isna().sum()

id          0
keyword     0
location    0
text        0
target      0
dtype: int64

In [12]:
df['input'] = 'KEY: ' + df.keyword + '; LOC: ' + df.location + '; TEXT: ' + df.text


In [13]:
df.input.head()

0    KEY: fatalities; LOC: USA; TEXT: Our Deeds are...
1    KEY: fatalities; LOC: USA; TEXT: Forest fire n...
2    KEY: fatalities; LOC: USA; TEXT: All residents...
3    KEY: fatalities; LOC: USA; TEXT: 13,000 people...
4    KEY: fatalities; LOC: USA; TEXT: Just got sent...
Name: input, dtype: object

In [14]:
from datasets import Dataset,DatasetDict

In [15]:
ds = Dataset.from_pandas(df)

In [16]:
ds

Dataset({
    features: ['id', 'keyword', 'location', 'text', 'target', 'input'],
    num_rows: 7613
})

In [17]:
model_nm = 'microsoft/deberta-v3-small'

In [18]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [19]:
tokz = AutoTokenizer.from_pretrained(model_nm)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
def tok_func(x): return tokz(x["input"])

In [21]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

In [22]:
tok_ds = tok_ds.rename_columns({'target':'labels'})

In [23]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5709
    })
    test: Dataset({
        features: ['id', 'keyword', 'location', 'text', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1904
    })
})

In [24]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe(include=[object])

,keyword,location,text
count,3237,2158,3263
unique,221,1602,3243
top,deluged,New York,11-Year-Old Boy Charged With Manslaughter of T...
freq,23,38,3


In [25]:
mode = eval_df.mode().iloc[0]

In [26]:
eval_df.fillna(mode,inplace=True)

In [27]:
eval_df.isna().sum()

id          0
keyword     0
location    0
text        0
dtype: int64

In [28]:
eval_df['input'] = 'KEY: ' + eval_df.keyword + '; LOC: ' + eval_df.location + '; TEXT: ' + eval_df.text
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

In [29]:
def corr(x,y): return np.corrcoef(x,y)[0][1]


In [30]:
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}


In [31]:
from transformers import TrainingArguments,Trainer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [32]:
bs = 128
epochs = 4
lr = 8e-5

In [33]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

ValueError: FP16 Mixed precision training with AMP or APEX (`--fp16`) and FP16 half precision evaluation (`--fp16_full_eval`) can only be used on CUDA devices.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

In [ ]:
trainer.train()

In [ ]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

In [ ]:
preds = np.clip(preds, 0, 1)


In [ ]:
preds #pred should be 1d list

In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)